# 第一课：连接阿里千问模型

## 学习目标
1. 了解如何配置阿里千问 API
2. 使用 LangChain 连接千问模型
3. 发送第一条消息并获取回复

## 背景知识

阿里千问（Qwen）支持两种调用方式：
- **DashScope SDK**：阿里云原生 SDK
- **OpenAI 兼容接口**：使用 OpenAI SDK 调用（本教程使用这种方式）

使用 OpenAI 兼容接口的好处是 LangChain 对其支持更好，代码更简洁。

## 1. 安装依赖

首先安装必要的包（如果已安装可以跳过）：

In [ ]:
# 安装依赖（只需运行一次）
!pip install langchain langchain-openai python-dotenv -q

## 2. 配置 API Key

你需要一个阿里云 DashScope 的 API Key。

获取地址：https://dashscope.console.aliyun.com/apiKey

**方式一**：直接在代码中设置（方便测试，但不推荐用于生产）

In [ ]:
import os

# 方式一：直接设置（替换为你的 API Key）
os.environ["DASHSCOPE_API_KEY"] = "your_api_key_here"

# 方式二：从 .env 文件加载（推荐）
# from dotenv import load_dotenv
# load_dotenv()

## 3. 创建千问聊天模型

使用 `ChatOpenAI` 连接千问模型，只需要修改 `base_url` 指向千问的 API 地址。

In [ ]:
from langchain_openai import ChatOpenAI

def create_qwen_chat():
    """
    创建千问聊天模型实例
    
    可选模型：
    - qwen-turbo: 速度快，适合简单任务
    - qwen-plus: 平衡性能和效果
    - qwen-max: 效果最好，适合复杂任务
    """
    api_key = os.getenv("DASHSCOPE_API_KEY")
    if not api_key or api_key == "your_api_key_here":
        raise ValueError("⚠️ 请设置你的 DASHSCOPE_API_KEY")
    
    chat = ChatOpenAI(
        model="qwen-plus",  # 模型名称
        openai_api_key=api_key,
        openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        temperature=0.7,  # 控制随机性，0-2，越高越随机
    )
    
    return chat

# 创建模型实例
chat = create_qwen_chat()
print("✅ 千问模型创建成功！")

## 4. 发送第一条消息

最简单的方式：直接传入字符串，调用 `invoke` 方法。

In [ ]:
# 发送一条简单的消息
response = chat.invoke("你好！请用一句话介绍一下你自己。")

print("问题: 你好！请用一句话介绍一下你自己。")
print(f"\n回复: {response.content}")
print(f"\n回复类型: {type(response).__name__}")

## 5. 使用消息类型

LangChain 支持多种消息类型：

| 类型 | 说明 |
|------|------|
| `SystemMessage` | 系统消息，设定 AI 的角色和行为 |
| `HumanMessage` | 用户消息 |
| `AIMessage` | AI 的回复 |

通过组合这些消息，可以更精确地控制对话。

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# 使用消息列表进行对话
messages = [
    SystemMessage(content="你是一个专业的 Python 编程助手，回答要简洁明了，用中文回答。"),
    HumanMessage(content="Python 的列表推导式是什么？给个简单例子。")
]

response = chat.invoke(messages)

print("系统设定: 你是一个专业的 Python 编程助手")
print("\n问题: Python 的列表推导式是什么？")
print(f"\n回复:\n{response.content}")

## 6. 模拟多轮对话

通过在消息列表中包含之前的对话，可以实现多轮对话效果。

In [ ]:
# 模拟多轮对话
messages = [
    SystemMessage(content="你是一个友好的助手。"),
    HumanMessage(content="我叫小明"),
    AIMessage(content="你好小明！很高兴认识你！有什么我可以帮助你的吗？"),
    HumanMessage(content="我叫什么名字？")  # AI 应该记得用户叫小明
]

response = chat.invoke(messages)

print("对话历史:")
print("  用户: 我叫小明")
print("  AI: 你好小明！很高兴认识你！")
print("  用户: 我叫什么名字？")
print(f"\nAI 回复: {response.content}")

## 7. 流式输出

对于长回复，流式输出可以实时显示内容，提升用户体验。

In [ ]:
print("问题: 用 Python 写一个快速排序的代码")
print("\n回复 (流式输出):")
print("-" * 40)

# 使用 stream 方法进行流式输出
for chunk in chat.stream("用 Python 写一个快速排序的代码，并添加简单注释"):
    print(chunk.content, end="", flush=True)

print("\n" + "-" * 40)

## 8. 练习

尝试修改下面的代码，让 AI 扮演不同的角色回答问题：

In [ ]:
# 练习：修改系统提示词，让 AI 扮演不同角色
# 例如：诗人、厨师、健身教练等

messages = [
    SystemMessage(content="你是一位经验丰富的厨师，擅长中餐。"),  # 修改这里的角色
    HumanMessage(content="今晚想做一道简单的家常菜，有什么推荐？")  # 修改问题
]

response = chat.invoke(messages)
print(response.content)

## 📝 本课小结

| 概念 | 说明 |
|------|------|
| `ChatOpenAI` | LangChain 的聊天模型接口 |
| `openai_api_base` | 指向千问 API 地址 |
| `invoke()` | 发送消息并获取回复 |
| `stream()` | 流式输出 |
| `SystemMessage` | 设定 AI 角色 |
| `HumanMessage` | 用户消息 |
| `AIMessage` | AI 回复 |

## ➡️ 下一课

下一课我们将学习 **Prompt Template（提示词模板）** 和 **Chain（链）**，让代码更加模块化和可复用。